In [1]:
#Import necessary libraries

import pandas as pd #data preprocessing
from bs4 import BeautifulSoup #web-scraping
import requests #HTTP/1.1 requests
import json #JSON encoder and decoder
import re #regular expression operations
from mysql.connector import connect, Error #database connector
print('Libraries imported')

Libraries imported


In [5]:
# Constant part of the website URL
url_const = 'https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title='

# User agent parameters
headers = {'headers': json.dumps({
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
             'Accept-Encoding': 'gzip, deflate',
             'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8',
             'Dnt': '1',
             'Host': 'httpbin.org',
             'Upgrade-Insecure-Requests': '1',
             'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) '
                           'AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/83.0.4103.97 Safari/537.36',
             'X-Amzn-Trace-Id': 'Root=1-5ee7bbec-779382315873aa33227a5df6'})
          }


In [ ]:
#Connect to the database
try:
    with connect(
        host='localhost',
        user='root',
        password='root',
        database="online_movie_rating",
    ) as connection:
        print(connection)
except Error as e:
    print(e)

In [6]:
years = ['2012-Q1', '2013-Q1', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

for year in years:
    url = url_const + year
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url, headers=headers).text

    # Parse HTML code for the entire site
    soup = BeautifulSoup(html_content, "lxml")
    
    # Find all tables on a page
    tables = soup.find_all('table')
    # Lets go ahead and scrape second table with HTML code tables[1]
    table1 = tables[1]
    # the head will form our column names
    body = table1.find_all("tr")
    # Head values (Column names) are the first items of the body list
    head = body[0] # 0th item is the header row
    body_rows = body[1:] # All other items becomes the rest of the rows

    # Lets now iterate through the head HTML code and make list of clean headings

    # Declare empty list to keep Columns names
    headings = []
    for item in head.find_all("th"): # loop through all th elements
        # convert the th elements to text and strip "\n"
        item = (item.text).rstrip("\n")
        # append the clean column name to headings
        headings.append(item)
    # Next is now to loop through the rest of the rows
    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will add entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
            # row_item.text removes the tags from the entries
            # the following regex is to remove \xa0 and \n and comma from row_item.text
            # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            #append aa to row - note one row entry is being appended
            row.append(aa)
        # append one row to all_rows
        all_rows.append(row)   
        # We can now use the data on all_rowsa and headings to make a table
    # all_rows becomes our data and headings the column names
    df = pd.DataFrame(data=all_rows,columns=headings)